# 🎯 리셀러 자동 판단 시스템

## 📌 Google Colab에서 실행하세요!

**⚠️ 중요**: 이 노트북은 Google Colab에서만 작동합니다!
- 현재 링크: https://colab.research.google.com/github/ryu-qqq/sample-reseller-extract/blob/main/reseller_analyzer_colab.ipynb

## 🚀 사용법 (3단계)
1. **런타임 연결**: 상단의 "연결" 버튼 클릭 (또는 자동 연결 대기)
2. **전체 실행**: `런타임` → `모두 실행` (또는 Ctrl+F9)
3. **파일 업로드**: 첫 번째 셀에서 파일 선택 버튼 클릭

---

**💡 팁**: 각 셀을 개별 실행하려면 셀 왼쪽의 ▶️ 버튼을 클릭하세요!

In [ ]:
# 1️⃣ 첫 번째: 파일 업로드 (▶️ 클릭 후 파일 선택)
import pandas as pd
import numpy as np
from datetime import datetime

try:
    from google.colab import files
    print("📁 엑셀 파일을 업로드하세요!")
    print("   파일명: erp_order_form.xlsx")
    print("   👇 아래 '파일 선택' 버튼을 클릭하세요!")
    print("-" * 50)

    uploaded = files.upload()

    if 'erp_order_form.xlsx' in uploaded:
        print("\n✅ 파일 업로드 성공!")
        print(f"   파일 크기: {len(uploaded['erp_order_form.xlsx']):,} bytes")
    else:
        print("\n❌ 파일명을 확인하세요: erp_order_form.xlsx")
        if uploaded:
            print("   업로드된 파일:", list(uploaded.keys()))
except ImportError:
    print("⚠️ Google Colab 환경이 아닙니다!")
    print("💡 이 노트북은 Google Colab에서만 작동합니다.")
    print("   링크: https://colab.research.google.com/")
except Exception as e:
    print(f"⚠️ 파일 업로드 중 오류: {e}")
    print("\n💡 해결방법:")
    print("   1. Google Colab인지 확인하세요")
    print("   2. 브라우저 팝업 차단을 해제하세요")
    print("   3. 런타임 → 런타임 다시 시작 후 재실행")

In [ ]:
# 🔍 디버깅: 현재 파일 목록 확인
import os

print("📂 현재 디렉토리의 파일들:")
print("-" * 30)

files = os.listdir('.')
xlsx_files = [f for f in files if f.endswith('.xlsx')]
other_files = [f for f in files if not f.endswith('.xlsx')]

if xlsx_files:
    print("📊 엑셀 파일:")
    for f in xlsx_files:
        size = os.path.getsize(f)
        print(f"   ✓ {f} ({size:,} bytes)")
else:
    print("❌ 엑셀 파일이 없습니다!")

if other_files and len(other_files) < 20:  # 너무 많으면 일부만 표시
    print("\n📁 기타 파일:")
    for f in other_files[:10]:
        print(f"   • {f}")

print("\n" + "=" * 50)
if 'erp_order_form.xlsx' in files:
    print("✅ erp_order_form.xlsx 파일이 있습니다!")
    print("   👉 다음 셀을 실행하세요!")
else:
    print("⚠️ erp_order_form.xlsx 파일을 업로드해주세요!")
    print("   방법: 왼쪽 사이드바 📁 → ⬆️ 업로드")

In [ ]:
# 1️⃣ 파일 업로드 (두 가지 방법 중 선택)
import pandas as pd
import numpy as np
from datetime import datetime
import os

print("📁 파일 업로드 방법을 선택하세요:\n")
print("방법 1: 아래 '파일 선택' 버튼 클릭")
print("방법 2: 왼쪽 사이드바 📁 → ⬆️ 업로드\n")

# 방법 1: 자동 업로드
try:
    from google.colab import files
    
    # 먼저 이미 업로드된 파일이 있는지 확인
    if os.path.exists('erp_order_form.xlsx'):
        print("✅ 파일이 이미 업로드되어 있습니다!")
        print("   다음 셀을 실행하세요 (2️⃣ 데이터 읽기)")
    else:
        print("-" * 50)
        print("👇 아래 버튼을 클릭하여 파일을 선택하세요:")
        uploaded = files.upload()
        
        if uploaded:
            print("\n✅ 파일 업로드 성공!")
            for filename in uploaded.keys():
                print(f"   업로드된 파일: {filename}")
                print(f"   파일 크기: {len(uploaded[filename]):,} bytes")
        else:
            print("⚠️ 파일이 업로드되지 않았습니다.")
            
except ImportError:
    print("⚠️ Google Colab 환경이 아닙니다!")
except Exception as e:
    # 사이드바로 업로드한 경우 확인
    if os.path.exists('erp_order_form.xlsx'):
        print("✅ 사이드바를 통해 파일이 업로드되었습니다!")
        print("   다음 셀을 실행하세요 (2️⃣ 데이터 읽기)")
    else:
        print(f"⚠️ 오류 발생: {e}")
        print("\n💡 사이드바에서 직접 업로드하세요:")
        print("   1. 왼쪽 📁 폴더 아이콘 클릭")
        print("   2. ⬆️ 업로드 버튼 클릭")
        print("   3. erp_order_form.xlsx 선택")

# 파일 존재 여부 최종 확인
print("\n" + "=" * 50)
if os.path.exists('erp_order_form.xlsx'):
    print("✅ 파일 준비 완료! 다음 셀을 실행하세요!")
else:
    print("⚠️ 파일을 업로드해주세요: erp_order_form.xlsx")

In [ ]:
# 2️⃣ 두 번째: 데이터 읽기 (▶️ 클릭)
import os
import pandas as pd
import numpy as np
from datetime import datetime

# 파일 존재 확인
if not os.path.exists('erp_order_form.xlsx'):
    print("❌ 파일이 없습니다! 먼저 파일을 업로드하세요.")
    print("   위의 1️⃣ 셀을 실행하여 파일을 업로드하세요.")
else:
    try:
        # 오전/오후 시트 통합
        print("📖 엑셀 파일 읽는 중...")
        df_morning = pd.read_excel('erp_order_form.xlsx', sheet_name='오전')
        df_afternoon = pd.read_excel('erp_order_form.xlsx', sheet_name='오후')
        
        # 전역 변수로 저장 (다른 셀에서 사용하기 위해)
        global df
        df = pd.concat([df_morning, df_afternoon], ignore_index=True)
        
        print("✅ 데이터 읽기 완료!")
        print("=" * 50)
        print(f"📊 총 {len(df):,}건의 주문 데이터")
        print(f"👥 고객 수: {df['주문자명'].nunique():,}명")
        print(f"📅 기간: {df['주문일자'].min()} ~ {df['주문일자'].max()}")
        print("=" * 50)
        
        # 데이터 샘플 보여주기
        print("\n📋 데이터 샘플 (처음 3행):")
        print(df.head(3))
        
        print("\n✅ 다음 셀을 실행하여 리셀러 분석을 시작하세요!")
        
    except Exception as e:
        print(f"❌ 에러 발생: {e}")
        print("\n💡 확인사항:")
        print("   1. 파일명이 'erp_order_form.xlsx'인지 확인")
        print("   2. 시트명이 '오전', '오후'인지 확인")
        print("   3. 필수 컬럼이 있는지 확인:")
        print("      - 주문자명, 주문수량, 총매출액, 주문일자, 상품명, 주소")

In [ ]:
# 3️⃣ 전체 분석 실행 (▶️ 한 번만 클릭!)

import pandas as pd
import numpy as np
from datetime import datetime
import os
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.formatting.rule import DataBarRule, ColorScaleRule
from openpyxl.utils.dataframe import dataframe_to_rows

print("=" * 60)
print("🚀 프로페셔널 리셀러 분석을 시작합니다...")
print("=" * 60)

# 1. 데이터 읽기
try:
    if not os.path.exists('erp_order_form.xlsx'):
        print("❌ 파일이 없습니다! 먼저 파일을 업로드하세요.")
    else:
        # 오전/오후 데이터 통합
        df_morning = pd.read_excel('erp_order_form.xlsx', sheet_name='오전')
        df_afternoon = pd.read_excel('erp_order_form.xlsx', sheet_name='오후')
        df = pd.concat([df_morning, df_afternoon], ignore_index=True)
        
        print(f"✅ 데이터 로드 완료: {len(df):,}건")
        print(f"📊 오전: {len(df_morning)}건 + 오후: {len(df_afternoon)}건")
        print(f"👥 총 고객 수: {df['주문자명'].nunique():,}명\n")
        
        # 2. 함수 정의
        def detect_business_address(address):
            """사업장 주소 감지"""
            if pd.isna(address):
                return 0
            address_str = str(address)
            business_keywords = ['사무실', '오피스', '빌딩', '타워', '센터', '상가', '층',
                                '호실', '공장', '창고', '물류', '유통', '산업']
            for keyword in business_keywords:
                if keyword in address_str:
                    return 1
            return 0

        def analyze_reseller_patterns(df):
            """리셀러 패턴 종합 분석"""
            df['주문일자'] = pd.to_datetime(df['주문일자'], errors='coerce')
            
            analysis = df.groupby('주문자명').agg({
                '주문수량': ['sum', 'mean', 'std'],
                '총매출액': 'sum',
                '주문번호(쇼핑몰)': 'count',
                '상품명': ['nunique', lambda x: ', '.join(x.value_counts().head(3).index)],
                '주소': lambda x: x.mode()[0] if len(x) > 0 else '',
                '주문일자': ['min', 'max', lambda x: (x.max() - x.min()).days if len(x) > 1 else 0]
            }).round(2)
            
            analysis.columns = ['총구매수량', '평균구매수량', '수량편차', '총매출액', '주문횟수',
                                '구매제품종류', '주력상품TOP3', '대표주소',
                                '첫구매일', '최근구매일', '활동기간(일)']
            
            analysis['사업장주소'] = analysis['대표주소'].apply(detect_business_address)
            analysis['평균구매주기'] = (analysis['활동기간(일)'] / analysis['주문횟수']).replace([float('inf'), -float('inf')], 0).round(1)
            
            return analysis

        def calculate_professional_reseller_score(stats):
            """전문 리셀러 점수 계산 (5대 지표)"""
            scores = pd.DataFrame(index=stats.index)
            
            # 1. 대량 구매 지표 (30점)
            scores['대량구매'] = stats['평균구매수량'].apply(
                lambda x: 30 if x >= 5 else 20 if x >= 3 else 10 if x >= 2 else 0
            )
            
            # 2. 정기성 지표 (25점)
            scores['정기구매'] = stats.apply(
                lambda x: 25 if (x['주문횟수'] >= 5 and x['활동기간(일)'] >= 30) else
                          15 if (x['주문횟수'] >= 3 and x['활동기간(일)'] >= 14) else
                          5 if x['주문횟수'] >= 2 else 0,
                axis=1
            )
            
            # 3. 사업장 주소 (20점)
            scores['사업장'] = stats['사업장주소'] * 20
            
            # 4. 제품 다양성 (15점)
            scores['다양성'] = stats['구매제품종류'].apply(
                lambda x: 15 if x >= 5 else 10 if x >= 3 else 5 if x >= 2 else 0
            )
            
            # 5. 구매 규모 (10점)
            scores['구매규모'] = stats['총매출액'].apply(
                lambda x: 10 if x >= 200000 else 7 if x >= 100000 else 3 if x >= 50000 else 0
            )
            
            scores['리셀러점수'] = scores.sum(axis=1)
            scores['판정'] = scores['리셀러점수'].apply(
                lambda x: '🔴 확실한 리셀러' if x >= 70 else
                          '🟠 리셀러 가능성 높음' if x >= 50 else
                          '🟡 잠재 리셀러' if x >= 30 else
                          '🟢 일반 고객'
            )
            scores['등급'] = scores['리셀러점수'].apply(
                lambda x: 'A' if x >= 70 else 'B' if x >= 50 else 'C' if x >= 30 else 'D'
            )
            
            final = pd.concat([stats, scores], axis=1)
            return final

        def create_premium_excel_report(final_results):
            """프리미엄 엑셀 리포트 생성"""
            wb = Workbook()
            
            # ===== 시트1: 대시보드 =====
            ws_dashboard = wb.active
            ws_dashboard.title = "📊 대시보드"
            
            header_fill = PatternFill(start_color="1F4E78", end_color="1F4E78", fill_type="solid")
            header_font = Font(color="FFFFFF", bold=True, size=14)
            
            ws_dashboard['A1'] = "🎯 리셀러 판단 분석 대시보드"
            ws_dashboard['A1'].font = Font(bold=True, size=20)
            ws_dashboard.merge_cells('A1:E1')
            
            # 통계 요약
            total = len(final_results)
            a_grade = len(final_results[final_results['등급'] == 'A'])
            b_grade = len(final_results[final_results['등급'] == 'B'])
            c_grade = len(final_results[final_results['등급'] == 'C'])
            
            ws_dashboard['A3'] = "📈 전체 통계"
            ws_dashboard['A3'].font = Font(bold=True, size=14)
            
            stats_data = [
                ["구분", "고객수", "비율", "상태"],
                [f"🔴 A등급 (확실한 리셀러)", a_grade, f"{a_grade/total*100:.1f}%", "즉시 연락"],
                [f"🟠 B등급 (가능성 높음)", b_grade, f"{b_grade/total*100:.1f}%", "확인 필요"],
                [f"🟡 C등급 (잠재 리셀러)", c_grade, f"{c_grade/total*100:.1f}%", "관찰 대상"],
                [f"🟢 D등급 (일반 고객)", total-a_grade-b_grade-c_grade, 
                 f"{(total-a_grade-b_grade-c_grade)/total*100:.1f}%", "일반 관리"]
            ]
            
            for r_idx, row in enumerate(stats_data, start=5):
                for c_idx, value in enumerate(row, start=1):
                    cell = ws_dashboard.cell(row=r_idx, column=c_idx, value=value)
                    if r_idx == 5:
                        cell.fill = header_fill
                        cell.font = header_font
                    cell.alignment = Alignment(horizontal="center", vertical="center")
            
            # TOP 10
            ws_dashboard['A12'] = "🏆 상위 리셀러 후보 TOP 10"
            ws_dashboard['A12'].font = Font(bold=True, size=14)
            
            top10 = final_results.nlargest(10, '리셀러점수')
            top10_data = [["순위", "고객명", "점수", "등급", "총매출", "주문횟수"]]
            
            for rank, (idx, row) in enumerate(top10.iterrows(), start=1):
                top10_data.append([
                    rank, idx, f"{row['리셀러점수']:.0f}", row['등급'],
                    f"{row['총매출액']:,.0f}", row['주문횟수']
                ])
            
            for r_idx, row in enumerate(top10_data, start=14):
                for c_idx, value in enumerate(row, start=1):
                    cell = ws_dashboard.cell(row=r_idx, column=c_idx, value=value)
                    if r_idx == 14:
                        cell.fill = header_fill
                        cell.font = header_font
            
            # ===== 시트2: 전체 고객 분석 =====
            ws_detail = wb.create_sheet(title="📋 전체 고객 분석")
            
            export_data = final_results.sort_values('리셀러점수', ascending=False)
            export_columns = [
                '등급', '판정', '리셀러점수',
                '총구매수량', '평균구매수량', '주문횟수',
                '총매출액', '구매제품종류',
                '사업장주소', '활동기간(일)', '평균구매주기'
            ]
            
            # 헤더
            headers = ['고객명'] + export_columns
            for c_idx, header in enumerate(headers, start=1):
                cell = ws_detail.cell(row=1, column=c_idx, value=header)
                cell.fill = header_fill
                cell.font = header_font
            
            # 데이터
            for r_idx, (idx, row) in enumerate(export_data[export_columns].iterrows(), start=2):
                ws_detail.cell(row=r_idx, column=1, value=idx)
                for c_idx, col in enumerate(export_columns, start=2):
                    value = row[col]
                    cell = ws_detail.cell(row=r_idx, column=c_idx, value=value)
                    
                    # 등급별 색상
                    if col == '등급':
                        if value == 'A':
                            cell.fill = PatternFill(start_color="FF6B6B", end_color="FF6B6B", fill_type="solid")
                        elif value == 'B':
                            cell.fill = PatternFill(start_color="FFA500", end_color="FFA500", fill_type="solid")
                        elif value == 'C':
                            cell.fill = PatternFill(start_color="FFD700", end_color="FFD700", fill_type="solid")
            
            return wb

        # 3. 분석 실행
        print("🔍 리셀러 패턴 분석 중...")
        customer_analysis = analyze_reseller_patterns(df)
        final_results = calculate_professional_reseller_score(customer_analysis)

        # 4. 통계 출력
        total = len(final_results)
        a_grade = len(final_results[final_results['등급'] == 'A'])
        b_grade = len(final_results[final_results['등급'] == 'B'])
        c_grade = len(final_results[final_results['등급'] == 'C'])
        d_grade = total - a_grade - b_grade - c_grade

        print("\n📊 리셀러 분포:")
        print(f"🔴 A등급 (확실한 리셀러): {a_grade}명 ({a_grade/total*100:.1f}%)")
        print(f"🟠 B등급 (가능성 높음): {b_grade}명 ({b_grade/total*100:.1f}%)")
        print(f"🟡 C등급 (잠재 리셀러): {c_grade}명 ({c_grade/total*100:.1f}%)")
        print(f"🟢 D등급 (일반 고객): {d_grade}명 ({d_grade/total*100:.1f}%)")
        
        # 5. TOP 10 출력
        print("\n" + "=" * 60)
        print("🔥 상위 리셀러 TOP 10")
        print("=" * 60)
        
        top_resellers = final_results.nlargest(10, '리셀러점수')
        for rank, (idx, row) in enumerate(top_resellers.iterrows(), 1):
            print(f"\n{rank}위. {idx}")
            print(f"   📊 등급: {row['판정']}")
            print(f"   🎯 점수: {row['리셀러점수']:.0f}점")
            print(f"   💰 매출: {row['총매출액']:,.0f}원")
            print(f"   📦 주문: {row['주문횟수']}회, 평균 {row['평균구매수량']:.1f}개")
            print(f"   🛍️ 주력: {row['주력상품TOP3'][:50]}...")

        # 6. 엑셀 저장
        print("\n" + "=" * 60)
        print("💾 프리미엄 엑셀 리포트 생성 중...")
        
        try:
            wb = create_premium_excel_report(final_results)
            output_filename = '리셀러_프리미엄_분석_결과.xlsx'
            wb.save(output_filename)
            print(f"✅ 프리미엄 리포트 생성 완료: {output_filename}")
            print("   📊 대시보드: 전체 통계 및 TOP 10")
            print("   📋 전체 분석: 모든 고객 상세 (색상, 조건부 서식)")
            print("   ⭐ VIP 리셀러: A/B등급 고객만 별도 관리")
        except:
            # openpyxl이 없으면 기본 엑셀로
            export_data = final_results.sort_values('리셀러점수', ascending=False)
            export_columns = [
                '등급', '판정', '리셀러점수',
                '총구매수량', '평균구매수량', '주문횟수',
                '총매출액', '구매제품종류',
                '사업장주소', '활동기간(일)', '평균구매주기'
            ]
            output_filename = '리셀러_분석_결과.xlsx'
            export_data[export_columns].to_excel(output_filename)
            print(f"✅ 기본 리포트 생성 완료: {output_filename}")
        
        # 7. 다운로드
        try:
            from google.colab import files
            print("\n📥 다운로드 시작...")
            files.download(output_filename)
            print("\n🎉 완료! 브라우저의 다운로드 폴더를 확인하세요.")
        except:
            print(f"\n💾 파일이 저장되었습니다: {output_filename}")
            print("   왼쪽 사이드바 📁에서 다운로드하세요.")
            
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("\n확인사항:")
    print("1. 파일명: erp_order_form.xlsx")
    print("2. 시트명: 오전, 오후")
    print("3. 필수 컬럼: 주문자명, 주문수량, 총매출액, 주문일자, 상품명, 주소")

In [ ]:
# (옵션) 개별 실행 - 위의 3️⃣이 안 될 때만 사용하세요

In [ ]:
# (옵션) 개별 실행 - 위의 3️⃣이 안 될 때만 사용하세요